In [1]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import structured_2d_vtk as idl_to_vtk

reload(idl_to_vtk)

<module 'structured_2d_vtk' from 'structured_2d_vtk.pyc'>

# Try to store a single time from Jason's IDL scripts
Store $\vec{B}$, $T$, and $n$.

In [2]:
b_x_measurement_points_timestamp = '115110949362'
b_y_measurement_points_timestamp = '115110949522'
b_z_measurement_points_timestamp = '115110938869'
triple_measurement_points_timestamp = '115112535917'
b_x_measurements_timestamp = '115112539367'
b_y_measurements_timestamp = '115112539384'
b_z_measurements_timestamp = '115112539331'
n_measurements_timestamp = '115112539443'
t_measurements_timestamp = '115112539427'

In [3]:
data_path = '../../cataloging_physics_quantities_in_2D/output/intermediate/'
data_ending = '.sav'
b_x_measurement_points_file = data_path + b_x_measurement_points_timestamp + data_ending
b_y_measurement_points_file = data_path + b_y_measurement_points_timestamp + data_ending
b_z_measurement_points_file = data_path + b_z_measurement_points_timestamp + data_ending
triple_measurement_points_file = data_path + triple_measurement_points_timestamp + data_ending 
b_x_measurements_file = data_path + b_x_measurements_timestamp + data_ending
b_y_measurements_file = data_path + b_y_measurements_timestamp + data_ending
b_z_measurements_file = data_path + b_z_measurements_timestamp + data_ending
n_measurements_file = data_path + n_measurements_timestamp + data_ending
t_measurements_file = data_path + t_measurements_timestamp + data_ending

In [4]:
b_x_measurement_points = idl.readsav(b_x_measurement_points_file)
b_y_measurement_points = idl.readsav(b_y_measurement_points_file)
b_z_measurement_points = idl.readsav(b_z_measurement_points_file)
triple_measurement_points =  idl.readsav(triple_measurement_points_file)
b_x_measurements = idl.readsav(b_x_measurements_file)
b_y_measurements = idl.readsav(b_y_measurements_file)
b_z_measurements = idl.readsav(b_z_measurements_file)
n_measurements = idl.readsav(n_measurements_file)
t_measurements = idl.readsav(t_measurements_file)

In [11]:
for time_point in xrange(21):
    file_name = '../output/2016-01-07/temperature_%i.vts' % time_point
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds([t_measurements])
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_scalar(t_measurements,
                                                  time_point,
                                                  x_min, x_max, y_min, y_max)
    vector_dict = {'a_out': t_resampled, 
                   'x_out': x_grid,
                   'y_out': y_grid}
    vector_dict = idl_to_vtk.remove_nans(vector_dict, time_point)
    print vector_dict['x_out']
    mesh = idl_to_vtk.prepare_mesh(vector_dict['x_out'], vector_dict['y_out'], 0.25)
    t_reshaped = idl_to_vtk.reshape_scalar(t_resampled)
    idl_to_vtk.write_to_structured_grid(file_name, t_reshaped, 't', mesh)

[-0.024      -0.02350505 -0.0230101  ...,  0.0240101   0.02450505  0.025     ]


IndexError: tuple index out of range

In [24]:
x_points = 20
y_points = 20
vector_dicts = [b_x_measurements, b_y_measurements, b_z_measurements]
(x_min, x_max, y_min, y_max) = idl_to_vtk.determine_sample_bounds(vector_dicts)
for time_point in [0, 1]:
    spline_x = idl_to_vtk.fit_bivariate_splines(vector_dicts[0], time_point,
                                                weigth=None, kx=1, ky=1,
                                                s=None)
    spline_y = idl_to_vtk.fit_bivariate_splines(vector_dicts[1], time_point,
                                                weigth=None, kx=1, ky=1,
                                                s=None)
    spline_z = idl_to_vtk.fit_bivariate_splines(vector_dicts[2], time_point,
                                                weigth=None, kx=1, ky=1,
                                                s=None)
    (vector_resampled_x,
     residual_x,
     x_grid,
     y_grid) = idl_to_vtk.evaluate_spline_on_structured_grid(spline_x,
                                                             x_min, x_max,
                                                             y_min, y_max,
                                                             x_points,
                                                             y_points)
    (vector_resampled_y,
     residual_y,
     x_grid,
     y_grid) = idl_to_vtk.evaluate_spline_on_structured_grid(spline_y,
                                                             x_min, x_max,
                                                             y_min, y_max,
                                                             x_points,
                                                             y_points)
    (vector_resampled_z,
     residual_z,
     x_grid,
     y_grid) = idl_to_vtk.evaluate_spline_on_structured_grid(spline_z,
                                                             x_min, x_max,
                                                             y_min, y_max,
                                                             x_points,
                                                             y_points)
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.025)
    vector = idl_to_vtk.reshape_vector(vector_resampled_x, vector_resampled_y,
                                       vector_resampled_z)
    print 'res_x', residual_x, 'res_y', residual_y, 'res_z', residual_z
    output_path = 'test' + '_%06i.vts' % time_point
    
    print vector.shape
    print vector[0].shape
    print mesh.shape
    
    
    
    #idl_to_vtk.write_to_structured_grid(output_path, vector,
    #                                    'B', mesh)

res_x 0.000187092125051 res_y 0.000253061837273 res_z 0.000153865222859
(3,)
(1, 20, 20, 1)
(3, 20, 20, 1)
res_x 0.000187018327353 res_y 0.000210235207473 res_z 0.000163585816182
(3,)
(1, 20, 20, 1)
(3, 20, 20, 1)
